In [1]:
import sys 
import os
sys.path.insert(0, "//home//nlpsomnath//NegNews//rebase//NN_Kafka//")
os.chdir("//home//nlpsomnath//NegNews//rebase//NN_Kafka//")
from nn_base.load_source_data import sqlpulldata
from nn_postprocessing import postprocess
from nn_schedulespiders import schedule_spiders, entity_jobs_finished
from kafka import KafkaProducer, KafkaConsumer, TopicPartition
from multiprocessing import Process
from scrapyd_api import ScrapydAPI
from datetime import datetime
import json
import nn_config as config 
import datetime

#mongo db 
import pymongo
import pandas as pd
from pymongo import MongoClient
import json

#load se object
from nn_base.nn_classes import load_results as lr 
from nn_base.nn_classes import search_entity as SE 

Using TensorFlow backend.


Configuration Parameters Loaded


### Collection to store spider status's (SPIDER STATUS TRACKER SETUP)

### Access Mongo Items and Converting to dataframe

In [2]:
#Access MongoDB Datastore 
#DB: negative_news_results 
#Collection: ResultData 
#Feilds: w2vSortedOuput[0]

#df = read_mongo(db, 'coll', query={}, host = 'localhost',port = 27017, username = None, password = None, no_id=True)
client = MongoClient(waitQueueTimeoutMS=1000)

#list all collections in each database (2 ways)
db_collections = dict((db, [collection for collection in client[db].collection_names()])
                      for db in client.database_names())

#print total rows in each collection withtin negative news
print([(collection,client['negative_news_results'][collection].count()) for collection in db_collections['negative_news_results']])    


[('ResultData', 118847)]


#### Aggregating Source Data 

In [3]:
#mongo client 
client = MongoClient(waitQueueTimeoutMS=100)

#access data om a database(2 ways shown below)
db_in = client['negative_news_results']

#list all collections in each database (2 ways)
db_collections = [collection for collection in db_in.collection_names()]
source_data = pd.DataFrame()
source_list=[]
print(db_collections)

#concatenate all collections into a list of dataframes, and aggregate the list of datafraemes into a single large dataframe
result_data = pd.concat([pd.DataFrame(list(db_in[collection].find({}))) for collection in db_collections])
result_data.sort_values(['Person','Run_ID'], ascending = False, kind = 'mergesort', inplace = True)
print('result data dimensions: {}*{}'.format(result_data.shape[0],result_data.shape[1]))

#result_data.to_pickle("/home/nlpsomnath/NegNews/zackc/results_df.pickle")

['ResultData']
result data dimensions: 118847*31


#### DATABASE STATISTICS 

#### DATAFRAME STATISTICS

In [9]:
df = result_data
People=list(df.groupby(['Person']).groups.keys())
EntriesPerPerson ={i:len(df.groupby(['Person']).groups[i]) for i in People}
#print(EntriesPerPerson)
list(df)
#summary stats by group across col (produces data.series)
#df.describe()
#df.groupby('entity')['entity'].count()
#df.groupby('entity')['date'].count()
#df.groupby('entity')['jobname'].count()

#summary of average sentiment per person data from source=bing 
#df[df['source'] == 'bing'].groupby('entity')['jobname']

#average similarity , sentiment, and Docscore per source for each person
#note:if you calculate more than one column of results, your result will 
#be a Dataframe. For a single column of results, the agg function, by default, will produce a Series.
stats = df.groupby(['Person','Doc'])['Similarity','Sentiment','FinalDocScore'].mean()
#df.groupby('Person', as_index=False).agg({"Similarity": "mean"})
#df['FinalEntityScore'].unique()
list(stats)

print(pd.Series(stats['Similarity']).agg('max') 
,pd.Series(stats['Similarity']).agg('min') 
,pd.Series(stats['Similarity']).quantile(q=.97)
,pd.Series(stats['Similarity']).quantile(q=.01)
      
     )

0.7455539452565352 0.11363522210792787 0.5587953117101965 0.20787045299278223


In [30]:
aggregation = {
    'Similarity':{
        'max_sim':'max',
        'avg_sim':'mean',
        'min_sim':'min',
        'std_sim':'std'
    },
    'Sentiment':{
        'max_sent':'max',
        'avg_sent':'mean',
        'min_sent':'min',
        'std_sent':'std'
    },
    'FinalDocScore':{
        'minDocScore':'min', 
        'maxDocScore':'max',
        'RangeDocScore': lambda x:max(x)-min(x)
    }
}

result_stats_person = pd.DataFrame(df.groupby('Person',as_index=False).agg(aggregation))
#list(result_stats_person)
#result_stats_person = df.agg(aggregation)
result_stats_person
#result_stats_total
#result_stats_person["FinalDocScore","maxDocScore"]

/usr/local/lib/python3.5/dist-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


TypeError: unorderable types: float() <= str()

In [10]:
import dateutil



#[result_data[var].nunique() for var in list(result_data) if isinstance(var,)]
#df['Column'] = list(df)
#df['TotalUnique'] =  [result_data[var].nunique() for var in list(result_data)]
#data.apply(lambda x: desm_pandas_wrapper(wv, q, x['content'], avg_top_n = 3), axis = 1)
#dataset shape
print('Data Columns: {}'.format(list(result_data)))
print('Total Rows:{}'.format(result_data.shape[0]))
print('Total Columns:{}'.format(result_data.shape[0]))
#unique_cnt(result_data)

#convert to datetime
#result_data['DateTime_Complete'] = result_data['DateTime_Complete'].apply(dateutil.parser.parse, dayfirst=True)

#final risk score 
#current: Avg(Sentence_Sim across all sentences)

#result_data['FinalDocScore'].min()
#result_data['FinalDocScore'].max()

#result_data.groupby(by=['Person'], as_index=False)['FinalDocScore'].mean()


#data = result_data[['Person','Doc','DocSent','DocSim',]]



#various pandas funcitons for summarizing
result_data.describe()
result_data['Similarity'].max()
result_data['Similarity'].mad()
result_data['Similarity'].mode()
result_data['Similarity'].median()
result_data['Similarity'].std()
result_data['Similarity'].var()
result_data['Similarity'].skew()
result_data['Similarity'].kurt()


result_data['Similarity'][result_data['source'] == 'bing'].max()
result_data['Person'].nunique()


Data Columns: ['DateTime_Complete', 'Doc', 'DocSim', 'FinalDocScore', 'FinalEntityScore', 'Final_Normalized_Score', 'JsonFileLocation', 'NewsSource', 'Person', 'RegNonReg', 'Run_ID', 'SQL_ID', 'Sent', 'Sentence', 'Sentiment', 'Similarity', 'Title', 'TotalLinks', 'URL', '_id', 'articleId', 'articleTitle', 'articleUrl', 'date', 'negativityScore', 'newsSource', 'pdftext', 'relevanceScore', 'source', 'summary', 'textsummary']
Total Rows:114991
Total Columns:114991


DateTime_Complete   Doc  DocSim  FinalDocScore FinalEntityScore  \
61453   11-05-18-15-15-17   0.0     NaN       0.590407            84.46   
61454   11-05-18-15-15-17   0.0     NaN       0.590407            84.46   
61455   11-05-18-15-15-17   0.0     NaN       0.590407            84.46   
61456   11-05-18-15-15-17   1.0     NaN       0.961053            84.46   
61457   11-05-18-15-15-17   1.0     NaN       0.961053            84.46   
61458   11-05-18-15-15-17   1.0     NaN       0.961053            84.46   
61459   11-05-18-15-15-17   2.0     NaN       0.870925            84.46   
61460   11-05-18-15-15-17   3.0     NaN       0.857665            84.46   
61461   11-05-18-15-15-17   3.0     NaN       0.857665            84.46   
61462   11-05-18-15-15-17   3.0     NaN       0.857665            84.46   
61463   11-05-18-15-15-17   3.0     NaN       0.857665            84.46   
61464   11-05-18-15-15-17   3.0     NaN       0.857665            84.46   
61465   11-05-18-15-15-17   3.0     NaN       0.857665            84.46   
61466   11-05-18-15-15-17   3.0     NaN       0.857665            84.46   
61467   11-05-18-15-15-17   4.0     NaN       0.888984            84.46   
61468   11-05-18-15-15-17   4.0     NaN       0.888984            84.46   
61469   11-05-18-15-15-17   4.0     NaN       0.888984            84.46   
61470   11-05-18-15-15-17   5.0     NaN       0.199397            84.46   
61471   11-05-18-15-15-17   5.0     NaN       0.199397            84.46   
61472   11-05-18-15-15-17   5.0     NaN       0.199397            84.46   
61473   11-05-18-15-15-17   5.0     NaN       0.199397            84.46   
61474   11-05-18-15-15-17   5.0     NaN       0.199397            84.46   
61475   11-05-18-15-15-17   5.0     NaN       0.199397            84.46   
61476   11-05-18-15-15-17   5.0     NaN       0.199397            84.46   
61477   11-05-18-15-15-17   6.0     NaN       1.000000            84.46   
61478   11-05-18-15-15-17   6.0     NaN       1.000000            84.46   
61479   11-05-18-15-15-17   6.0     NaN       1.000000            84.46   
61480   11-05-18-15-15-17   6.0     NaN       1.000000            84.46   
61481   11-05-18-15-15-17   6.0     NaN       1.000000            84.46   
61482   11-05-18-15-15-17   6.0     NaN       1.000000            84.46   
...                   ...   ...     ...            ...              ...   
106043  11-27-18-20-03-02   8.0     NaN       0.615090           109.75   
106044  11-27-18-20-03-02   8.0     NaN       0.615090           109.75   
106045  11-27-18-20-03-02   9.0     NaN       0.312171           109.75   
106046  11-27-18-20-03-02   9.0     NaN       0.312171           109.75   
106047  11-27-18-20-03-02   9.0     NaN       0.312171           109.75   
106048  11-27-18-20-03-02   9.0     NaN       0.312171           109.75   
106049  11-27-18-20-03-02   9.0     NaN       0.312171           109.75   
106050  11-27-18-20-03-02   9.0     NaN       0.312171           109.75   
106051  11-27-18-20-03-02   9.0     NaN       0.312171           109.75   
106052  11-27-18-20-03-02   9.0     NaN       0.312171           109.75   
106053  11-27-18-20-03-02   9.0     NaN       0.312171           109.75   
106054  11-27-18-20-03-02   9.0     NaN       0.312171           109.75   
106055  11-27-18-20-03-02   9.0     NaN       0.312171           109.75   
106056  11-27-18-20-03-02   9.0     NaN       0.312171           109.75   
106057  11-27-18-20-03-02   9.0     NaN       0.312171           109.75   
106058  11-27-18-20-03-02  10.0     NaN       0.664231           109.75   
106059  11-27-18-20-03-02  10.0     NaN       0.664231           109.75   
106060  11-27-18-20-03-02  10.0     NaN       0.664231           109.75   
106061  11-27-18-20-03-02  10.0     NaN       0.664231           109.75   
106062  11-27-18-20-03-02  10.0     NaN       0.664231           109.75   
106063  11-27-18-20-03-02  10.0     NaN       0.664231           109.75   
106064  11-27-18-20-03-02  11.0  

#### converting data to and from json records
#### Note: Must drop _id column 

In [ ]:
try:  
    result_data.drop(['_id'], axis=1, inplace=True)
except:
    pass 


json_result = result_data.to_json(orient='records')
df_result = pd.read_json(json_result)

#print(df_result.head(4))
list(source_data)
#split json output into serires , by column 
#[result_data[s].to_json(orient = 'split') for s in result_data.columns ]
#print(source_data['jobname'].unique() , df_result['Run_ID'].unique())
#pd.set_option('max_colwidth',40000)
#df_stats_subset = source_data[(source_data['entity']=='UBS') & (source_data['source']=='occ')]['content']
#df_stats_subset

In [ ]:
#Target Data: Output SE Object 
se_list,df = lr(nn_class_path ="//home//docadmin//Mueller//DTCC_Img//DTCC-demo//", folderpath = "//home//docadmin//Mueller//DTCC_Img//DTCC-demo//output_pdf//")  

#2.Check who we ran and select oj simpson for test 
namelist = [se_list[i].entity_fname for i in range(len(se_list))]
#se = [se_list[i] for i in range(len(se_list)) if se_list[i].entity_fname == 'OJ Simpson'][0]
se = [se_list[i] for i in range(len(se_list)) ][0]
print(list(vars(se)))
#print(se.entity_name)




sent_df =pd.read_json(se.w2vDfSorted[0],orient='index')
doc_df = sent_df.groupby(by=['Person','Doc','URL'],as_index=False)['Similarity','Sentiment'].mean()# ; doc_level.fillna(0, inplace=True)
doc_df.rename(index=str, columns={"Similarity": "DocSim", "Sentiment": "DocSent"}, inplace=True)

#doc_df['DocSim'] = doc_df['Similarity']
#doc_df['DocSent'] = doc_df['Sentiment']
#docdf = doc_df.drop(['Sentiment','Similarity'], axis=1)
df_merge = sent_df.merge(doc_df[['Doc','DocSim','DocSent']], how='left',left_on='Doc',right_on='Doc')    
df_merge.sort_values(['FinalDocScore','DocSim'], ascending = False, kind = 'mergesort', inplace = True)
df_merge.rename(index=str, columns={"Doc": "articleId", "URL": "articleUrl","Title":"articleTitle","FinalDocScore":"relevanceScore","DocSent":"negativityScore","NewsSource":"newsSource","textsummary":"summary"}, inplace=True)


df_final=df_merge[['articleId'
          , 'articleUrl'
          , 'articleTitle'
          , 'summary'
          , 'relevanceScore'
          , 'negativityScore'
          , 'newsSource'
          , 'date'
         ]]



len(se.LSA_filter_count)
len(se.urllist[0])


import ast
ast.literal_eval(df_final.to_json(orient="records"))

In [ ]:
import pytz

local_tz = pytz.timezone('UTC') # use your local timezone name here
# NOTE: pytz.reference.LocalTimezone() would produce wrong result here

## You could use `tzlocal` module to get local timezone on Unix and Win32
# from tzlocal import get_localzone # $ pip install tzlocal

# # get local timezone    
# local_tz = get_localzone()

def utc_to_local(utc_dt):
    local_dt = utc_dt.replace(tzinfo=pytz.utc).astimezone(local_tz)
    return local_tz.normalize(local_dt) # .normalize might be unnecessary

from datetime import datetime
datetime.utcnow()

### Posting to MongoDB 

In [ ]:
#sort and create index row 
df =pd.read_json(se.w2vDfSorted[0],orient='index')
df.sort_values(['FinalDocScore','Similarity'], ascending = False, kind = 'mergesort', inplace = True)

#post to mongo
db = client.negative_news_results.ResultData
write_df_to_mongoDB(  df,database_name = 'negative_news_results' ,
                          collection_name = 'ResultData',
                          server = 'localhost',
                          mongodb_port = 27017,
                          chunk_size = 3)

### Format of contract response (Not Pushed to MongoDB , Pushed to Screen) 

In [ ]:
import time
record = {
    'Run_ID': se.entity_fname.replace(' ','_')+'_'.replace('__','_')+se.entity_lname+se.entity_id.replace('__','_').replace(' ','') 
    , 'Entity_Name': se.entity_fname +' '+se.entity_lname
    , 'DateTime_Complete': time.strftime("%Y-%m-%d-%H-%M-%S")
    , 'OverallRiskScore': se.riskscore_final[0]
    , 'w2vSortedOutput': se.w2vDfSorted[0]
}


In [ ]:
#add a new collection to the negative news database 

#0.define client and database you want to check for collection existance
client = MongoClient()
db = client['negative_news_results']

#1.post data to collection if it exists
if 'ResultData' in db.collection_names():
    post = {
    "url":"zacks.test.url"
    , "entity":"zack carideo"
    , "source":"zacks house"
    , "date":datetime.datetime.utcnow()
    , "title":"Zacks Test Title"
    , "jobname":"09-25-29-23-59-59"
    , "origtext": "test original text"
    , "content": "test original content"
    , "summary":"test summary"
    }
    print('collection found, posting data')
    data = db.ResultData.insert_one(post).inserted_id

else: 
    print('ResultData not a colletion')


#### Function to post many records to mongodb from a dataframe at once

In [ ]:
def write_df_to_mongoDB(  my_df,\
                          database_name = 'mydatabasename' ,\
                          collection_name = 'mycollectionname',
                          server = 'localhost',\
                          mongodb_port = 27017,\
                          chunk_size = 100):
    #"""
    #This function take a list and create a collection in MongoDB (you should
    #provide the database name, collection, port to connect to the remoete database,
    #server of the remote database, local port to tunnel to the other machine)
    #
    #---------------------------------------------------------------------------
    #Parameters / Input
    #    my_list: the list to send to MongoDB
    #    database_name:  database name
    #
    #    collection_name: collection name (to create)
    #    server: the server of where the MongoDB database is hosted
    #        Example: server = '132.434.63.86'
    #    this_machine_port: local machine port.
    #        For example: this_machine_port = '27017'
    #    remote_port: the port where the database is operating
    #        For example: remote_port = '27017'
    #    chunk_size: The number of items of the list that will be send at the
    #        some time to the database. Default is 100.
    #
    #Output
    #    When finished will print "Done"
    #----------------------------------------------------------------------------
    #FUTURE modifications.
    #1. Write to SQL
    #2. Write to csv
    #----------------------------------------------------------------------------
   
    #"""



    #To connect
    # import os
    # import pandas as pd
    # import pymongo
    # from pymongo import MongoClient

    client = MongoClient('localhost',int(mongodb_port))
    db = client[database_name]
    collection = db[collection_name]
    # To write
    #collection.delete_many({})  # Destroy the collection
    #aux_df=aux_df.drop_duplicates(subset=None, keep='last') # To avoid repetitions
    my_list = my_df.to_dict('records')
    
    # Insert the dataframe
    collection.insert_many(my_list) # fill de collection


    print('Done')
    return

### Pymongo aggregation pipelines <br>http://api.mongodb.com/python/current/examples/aggregation.html
    

In [ ]:
startgroup = db.newsapi_search.aggregate(
# The Aggregation Pipeline is defined as an array of different operations
[
    # The first stage in this pipe is to group data
    {
        '$group':
            {
                '_id':"$entity",
                "count":
                        {'$sum':1}
            }
    }, 
    
    #second stage in pipe line sorts the data 
    {"$sort": {"_id":1}
}
# Close the array with the ] tag 
])

#print results 
for group in startgroup:
    print(group)

#### Insert a document into a collection 

In [ ]:
#Note: the _id column is automatically added to the document when it is posted to the MongoDB Database
#Note that documents can contain native Python types (like datetime.datetime instances) which will be
#automatically converted to and from the appropriate BSON types.
post = {
    "url":"zacks.test.url"
    , "entity":"zack carideo"
    , "source":"zacks house"
    , "date":datetime.datetime.utcnow()
    , "title":"Zacks Test Title"
    , "jobname":"09-25-29-23-59-59"
    , "origtext": "test original text"
    , "content": "test original content"
    , "summary":"test summary"
}

data = client.negative_news.newsapi_search.insert_one(post).inserted_id
data

In [ ]:
dict_keys(['url', 'entity', 'source', 'date', 'title', 'jobname', 'origtext', 'content', 'summary', '_id'])

### Delete Data from Collections 


In [ ]:
#mongo client 
client = MongoClient(waitQueueTimeoutMS=100)

#access data om a database(2 ways shown below)
#db = client['negative_news']
db_in = client['negative_news']
db_out = client['negative_news_results']

#delete all source data 
db_in.bing_search.delete_many({})
db_in.regulatory_news_search.delete_many({})
db_in.newsapi_search.delete_many({})

#delete all results 
db_out.ResultData.delete_many({})

#### NEXT STEP: LINKING SOURCE DATA FROM BULK PULL TO OUTPUT TABLE USING THE SQL SPECIFIC ID COLUMN 

In [ ]:
#Source Data: SQL TABLE 
sqldata=sqlpulldata(config.connection_string, config.start_date , config.end_date , config.datevar)

In [ ]:
strg = {"{\'entityName\': \'Gibraltar Private Bank \', \'DateTime_Complete\': \'10-11-18-11-31-49\', \'JsonFileLocation\': \'/home/docadmin/Mueller/DTCC_Img/DTCC-demo/output_pdf/10-11-18-11-30-09_Gibraltar_Private_Bank_10-11-2018.json\', \'linksRemainingAfterFilter\': 20,'entityId\': 31241, \'aaEntityId\': 0}"}
type(strg)



#### FUNCTIONS 


In [ ]:
def myprint(d):
    for k, v in d.items():
        if isinstance(v, dict):
            myprint(v)
        else:
            print("{0} : {1}".format(k, v))
            
#unique values in each col
def unique_cnt(df):
    coldic={col:len(df[col].value_counts()) for col in list(df)}
    return coldic

In [7]:
import pickle
import pandas
    
#pickle_off = open(r"/home/nlpsomnath/NegNews/rebase/NN_Kafka/output/consolidated_results_11-28-18-13-13-58.pk", "rb")
#emp = pickle.load(pickle_off)
#print(emp)

# we open the file for reading
with open("/home/nlpsomnath/NegNews/rebase/NN_Kafka/output/consolidated_results_11-28-18-13-53-45.pkl",'rb') as fileObject:
        pickle.load(fileObject)
# load the object from the file into var b
#b = pandas.read_pickle(fileObject)  

FileNotFoundError: [Errno 2] No such file or directory